In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import requests
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}

In [62]:
# List of state abbreviations
states = ['AL', 'AK', 'AZ' ,'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
 'NH', 'NJ', 'NM', 'NY', 'DC', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [75]:
full = pd.DataFrame()
for state in states:
   
    curr = pd.read_csv(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.csv?measureid=MHLTH&$limit=4000&stateabbr={state}',nrows=4000)
    full.append(curr)
   

KeyboardInterrupt: 

In [132]:
list = []
x = 0
#df = pd.json_normalize(res.json())
for state in states:
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=MHLTH&$limit=4000&stateabbr={state}')
    list.append(res.json())
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=DEPRESSION&$limit=4000&stateabbr={state}')
    list.append(res.json())
    
    

In [133]:
df = pd.json_normalize(list[0])
df.set_index('locationid')
for i in range(1,len(list)):
    curr = pd.json_normalize(list[i])
    df = df.append(curr)
df

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,...,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation.type,geolocation.coordinates,data_value_footnote_symbol,data_value_footnote
0,2019,AL,Alabama,Childersburg,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,21.7,...,5175,0114464,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health,Point,"[-86.3516749, 33.27933748]",NaN,NaN
1,2019,AL,Alabama,Broomtown,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,19.8,...,182,0109880,HLTHSTAT,MHLTH,CrdPrv,Mental Health,Point,"[-85.5193995, 34.36265501]",NaN,NaN
2,2019,AL,Alabama,Flomaton,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,20.8,...,1440,0126824,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health,Point,"[-87.25419172, 31.01266438]",NaN,NaN
3,2019,AL,Alabama,Center Point,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,19.1,...,16921,0113264,HLTHSTAT,MHLTH,CrdPrv,Mental Health,Point,"[-86.68355223, 33.64307991]",NaN,NaN
4,2019,AL,Alabama,Ashville,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,20.2,...,2212,0102908,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health,Point,"[-86.26978035, 33.83311158]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2019,WY,Wyoming,Green River,BRFSS,Health Outcomes,Depression among adults aged >=18 years,%,Age-adjusted prevalence,19.1,...,12515,5633740,HLTHOUT,DEPRESSION,AgeAdjPrv,Depression,Point,"[-109.4697692, 41.51506709]",NaN,NaN
360,2019,WY,Wyoming,Hill View Heights,BRFSS,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,18.0,...,170,5637727,HLTHOUT,DEPRESSION,CrdPrv,Depression,Point,"[-104.1516713, 43.81587217]",NaN,NaN
361,2019,WY,Wyoming,Boulder,BRFSS,Health Outcomes,Depression among adults aged >=18 years,%,Age-adjusted prevalence,17.6,...,170,5609235,HLTHOUT,DEPRESSION,AgeAdjPrv,Depression,Point,"[-109.6979738, 42.74292959]",NaN,NaN
362,2019,WY,Wyoming,Bairoil,BRFSS,Health Outcomes,Depression among adults aged >=18 years,%,Age-adjusted prevalence,NaN,...,106,5604885,HLTHOUT,DEPRESSION,AgeAdjPrv,Depression,Point,"[-107.561048, 42.23720184]",#,Not estimated due to lack of population sizes ...


In [130]:
health_data_depr = pd.read_csv('https://chronicdata.cdc.gov/resource/eav7-hnsx.csv?measureid=DEPRESSION',nrows=999999)
health_data_mtl = pd.read_csv('https://chronicdata.cdc.gov/resource/eav7-hnsx.csv?measureid=MHLTH&$limit=4000&stateabbr=AL',nrows=4000)
full = pd.read_csv('PLACES__Local_Data_for_Better_Health__Place_Data_2021_release.csv')
full

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,...,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text
0,2019,AL,Alabama,Carlton,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,Age-adjusted prevalence,5.9,...,NaN,5.3,6.6,65,POINT (-87.8385235 31.34655389),112112,HLTHOUT,STROKE,AgeAdjPrv,Stroke
1,2019,AL,Alabama,Chatom,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,40.8,...,NaN,39.5,42.0,1288,POINT (-88.24602114 31.46914557),114008,HLTHOUT,OBESITY,CrdPrv,Obesity
2,2019,AL,Alabama,Collinsville,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.3,...,NaN,4.7,6.0,1983,POINT (-85.865567 34.26230655),116600,HLTHOUT,STROKE,CrdPrv,Stroke
3,2019,AL,Alabama,Colony,BRFSS,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,19.4,...,NaN,18.2,20.9,268,POINT (-86.8939975 33.9372899),116684,HLTHOUT,DEPRESSION,CrdPrv,Depression
4,2019,AL,Alabama,Coosada,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,4.9,...,NaN,4.6,5.3,1226,POINT (-86.3195555 32.49141923),117176,HLTHOUT,STROKE,CrdPrv,Stroke
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685021,2019,WI,Wisconsin,Tunnel City,BRFSS,Prevention,Visits to doctor for routine checkup within th...,%,Age-adjusted prevalence,73.3,...,NaN,72.0,74.4,106,POINT (-90.56797911 44.00495928),5581025,PREVENT,CHECKUP,AgeAdjPrv,Annual Checkup
1685022,2018,WY,Wyoming,Burlington,BRFSS,Prevention,Visits to dentist or dental clinic among adult...,%,Crude prevalence,58.0,...,NaN,53.5,62.1,288,POINT (-108.432023 44.44706563),5611120,PREVENT,DENTAL,CrdPrv,Dental Visit
1685023,2019,WY,Wyoming,Buffalo,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,9.1,...,NaN,8.6,9.7,4586,POINT (-106.7149007 44.34210256),5610685,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma
1685024,2019,WI,Wisconsin,Waupun,BRFSS,Prevention,Visits to doctor for routine checkup within th...,%,Crude prevalence,72.9,...,NaN,71.9,73.8,11357,POINT (-88.73778331 43.63160974),5584425,PREVENT,CHECKUP,CrdPrv,Annual Checkup


In [131]:
full.loc[full['MeasureId'] == 'MHLTH']

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,...,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,Geolocation,LocationID,CategoryID,MeasureId,DataValueTypeID,Short_Question_Text
1134,2019,AL,Alabama,Childersburg,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,21.7,...,NaN,20.4,23.0,5175,POINT (-86.3516749 33.27933748),114464,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health
1151,2019,AL,Alabama,Broomtown,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,19.8,...,NaN,18.0,21.5,182,POINT (-85.5193995 34.36265501),109880,HLTHSTAT,MHLTH,CrdPrv,Mental Health
1178,2019,AL,Alabama,Flomaton,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,20.8,...,NaN,19.5,22.1,1440,POINT (-87.25419172 31.01266438),126824,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health
1224,2019,AL,Alabama,Center Point,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,19.1,...,NaN,18.5,19.7,16921,POINT (-86.68355223 33.64307991),113264,HLTHSTAT,MHLTH,CrdPrv,Mental Health
1228,2019,AL,Alabama,Ashville,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,20.2,...,NaN,19.0,21.4,2212,POINT (-86.26978035 33.83311158),102908,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684972,2019,WY,Wyoming,Bear River,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,15.3,...,NaN,14.1,16.7,612,POINT (-111.03572 41.37997397),5605625,HLTHSTAT,MHLTH,CrdPrv,Mental Health
1684988,2019,WY,Wyoming,Guernsey,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,13.4,...,NaN,12.3,14.6,1147,POINT (-104.7449408 42.26846479),5634320,HLTHSTAT,MHLTH,CrdPrv,Mental Health
1684991,2019,WI,Wisconsin,Trego,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,14.2,...,NaN,12.7,15.7,227,POINT (-91.82870776 45.90485559),5580425,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health
1684992,2019,WY,Wyoming,Turnerville,BRFSS,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,13.3,...,NaN,12.0,14.7,192,POINT (-110.9007179 42.86226801),5678400,HLTHSTAT,MHLTH,AgeAdjPrv,Mental Health


In [2]:
# Import Income Data Set
income_df = pd.read_csv("kaggle_income.csv", encoding='latin-1')

# delete unnecessary columns
del income_df['id']
del income_df['Type']
del income_df['County']
del income_df['Place']
del income_df['Primary']
del income_df['Lat']
del income_df['Lon']
del income_df['sum_w']

# combine duplicate cities
income_df.head(10)

,State_Code,State_Name,State_ab,City,Zip_Code,Area_Code,ALand,AWater,Mean,Median,Stdev
0,1,Alabama,AL,Chickasaw,36611,251,10894952,909156,38773,30506,33101
1,1,Alabama,AL,Louisville,36048,334,26070325,23254,37725,19528,43789
2,1,Alabama,AL,Columbiana,35051,205,44835274,261034,54606,31930,57348
3,1,Alabama,AL,Satsuma,36572,251,36878729,2374530,63919,52814,47707
4,1,Alabama,AL,Dauphin Island,36528,251,16204185,413605152,77948,67225,54270
5,1,Alabama,AL,Cullman,35057,256,8913021,26837,50715,42643,35886
6,1,Alabama,AL,East Brewton,36426,251,8826252,91015,33737,23610,28256
7,1,Alabama,AL,Coosada,36020,334,10222339,176500,46319,40242,38941
8,1,Alabama,AL,Eva,35621,256,10544874,78981,57994,39591,47235
9,1,Alabama,AL,Sylacauga,35151,256,45178321,6034534,54807,41712,51359


In [23]:
# import us cities dataset
cities_df = pd.read_csv('uscities.csv')
del cities_df['city_ascii']
del cities_df['county_name']
del cities_df['lat']
del cities_df['lng']
del cities_df['source']
del cities_df['military']
del cities_df['incorporated']
del cities_df['timezone']
del cities_df['ranking']
del cities_df['id']
cities_df.head(10)
print(cities_df.loc[(cities_df['city'] == 'Rockville')])

            city state_id      state_name  county_fips  population  density  \
713    Rockville       MD        Maryland        24031       68079     1933   
5344   Rockville       CT     Connecticut         9013        7115     1624   
7462   Rockville       IN         Indiana        18121        4124      641   
9605   Rockville       MN       Minnesota        27145        2585       35   
22303  Rockville       UT            Utah        49053         269       12   
24708  Rockville       MO        Missouri        29013         157      184   
25236  Rockville       SC  South Carolina        45019         136      123   
26204  Rockville       NE        Nebraska        31163         100      173   
28223  Rockville       AL         Alabama         1025          12        1   

                                            zips  
713    20852 20850 20851 20847 20848 20849 20857  
5344                                       06066  
7462                                       47872  
9605 